# Links úteis para se usar datas para treinar modelo 
https://towardsdatascience.com/machine-learning-with-datetime-feature-engineering-predicting-healthcare-appointment-no-shows-5e4ca3a85f96

https://stackoverflow.com/questions/16453644/regression-with-date-variable-using-scikit-learn


### Para treinar o modelo fazer onehot encoding para distritos e concelhos e ignorar as localidas (para não ficarmos com um nº ridiculo de colunas)

In [28]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [29]:
data = pd.read_csv('data/anpc_final.csv')

/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573971 entries, 0 to 573970
Data columns (total 15 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   Numero                                  573971 non-null  object 
 1   DataOcorrencia                          573966 non-null  object 
 2   DataFechoOperacional                    418785 non-null  object 
 3   Natureza                                573966 non-null  object 
 4   EstadoOcorrencia                        573966 non-null  object 
 5   Distrito                                573966 non-null  object 
 6   Concelho                                573966 non-null  object 
 7   Freguesia                               573964 non-null  object 
 8   Localidade                              573964 non-null  object 
 9   Latitude                                573962 non-null  object 
 10  Longitude                               5739

In [31]:
data.describe()

,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
count,573962.000000,573962.000000,573962.000000,573962.000000
mean,1.522423,4.114807,0.018506,0.025385
std,2.303219,8.293575,0.220632,0.351961
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,0.000000
75%,1.000000,4.000000,0.000000,0.000000
max,354.000000,2170.000000,41.000000,61.000000


In [66]:
data.head(3)

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
0,2016150002576,2016-09-01 14:02:00,2016-09-01 17:30:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SETÚBAL,MOITA,Gaio-Rosário e Sarilhos Pequenos,Gaio- Rosario,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0,0 days 03:28:00
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0,NaT
2,2016020002249,2016-01-24 18:57:00,2016-01-24 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0,0 days 00:56:00


In [33]:
#Número de ocorencias por distrito

data.groupby(['Distrito']).size().sort_values(ascending=False)

Distrito
LISBOA              97695
PORTO               93203
BRAGA               42017
SETÚBAL             41425
AVEIRO              41361
SANTARÉM            33464
VISEU               30291
FARO                29947
LEIRIA              29685
COIMBRA             25495
VILA REAL           17934
BRAGANÇA            15863
VIANA DO CASTELO    15282
GUARDA              13423
CASTELO BRANCO      12912
BEJA                12011
PORTALEGRE          11550
ÉVORA               10406
0                       2
dtype: int64

In [48]:
data.Distrito.isna().sum()

0

In [34]:
data[data['Distrito'] == '0']

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
83938,LAR SAO JOSE,"39,5983240486353","-8,39819137470253",1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262081,Travessa da Água da Flor - Frent PSP,"38,71853771","-9,165754765",1,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#Retirar ocorrências sem distrito definido
data.drop(data[data['Distrito'] == '0'].index, inplace=True)

In [36]:
data.groupby(['Distrito']).size().sort_values(ascending=False)

Distrito
LISBOA              97695
PORTO               93203
BRAGA               42017
SETÚBAL             41425
AVEIRO              41361
SANTARÉM            33464
VISEU               30291
FARO                29947
LEIRIA              29685
COIMBRA             25495
VILA REAL           17934
BRAGANÇA            15863
VIANA DO CASTELO    15282
GUARDA              13423
CASTELO BRANCO      12912
BEJA                12011
PORTALEGRE          11550
ÉVORA               10406
dtype: int64

In [37]:
##HISTOGRAMA NÚMERO DE OCORRêNCIAS POR DISTRITO## 

In [38]:
# Mostrar ocorrências por distrito num mapa (ideia possível e que o sérgio vai gostar, mas temos que discutir isto melhor)

In [39]:
data.DataOcorrencia = pd.to_datetime(data.DataOcorrencia)
data.DataOcorrencia.dt.year.unique()

array([2016.,   nan, 2017., 2018.])

In [41]:
data.DataFechoOperacional = pd.to_datetime(data.DataFechoOperacional)

In [42]:
data.head()

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos
0,2016150002576,2016-09-01 14:02:00,2016-09-01 17:30:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SETÚBAL,MOITA,Gaio-Rosário e Sarilhos Pequenos,Gaio- Rosario,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0
2,2016020002249,2016-01-24 18:57:00,2016-01-24 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0
3,2016050041560,2016-01-30 23:38:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,CASTELO BRANCO,VILA DE REI,Vila de Rei,Vila de Rei,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0
4,2016020003825,2016-02-13 00:10:00,2016-02-13 01:35:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Santa Bárbara de Padrões,"SETE-Rua da Graça, 4","37,6149714","-7,962115173",1.0,2.0,0.0,0.0


In [69]:
# Creating the column representing the ammout of time necessary to solve each occurrence
data['TempoResolucao'] = (data.DataFechoOperacional - data.DataOcorrencia)
data.head()

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
0,2016150002576,2016-09-01 14:02:00,2016-09-01 17:30:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SETÚBAL,MOITA,Gaio-Rosário e Sarilhos Pequenos,Gaio- Rosario,"38,68091202","-9,002235449",1.0,3.0,0.0,0.0,0 days 03:28:00
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0,NaT
2,2016020002249,2016-01-24 18:57:00,2016-01-24 19:53:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Castro Verde e Casével,"CASTRO VERDE-Rua Bento J. Carraça,6","37,71278977","-8,091107843",2.0,4.0,0.0,0.0,0 days 00:56:00
3,2016050041560,2016-01-30 23:38:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,CASTELO BRANCO,VILA DE REI,Vila de Rei,Vila de Rei,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0,NaT
4,2016020003825,2016-02-13 00:10:00,2016-02-13 01:35:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,BEJA,CASTRO VERDE,Santa Bárbara de Padrões,"SETE-Rua da Graça, 4","37,6149714","-7,962115173",1.0,2.0,0.0,0.0,0 days 01:25:00


In [76]:
# Checking for negative resolution times
data[data.TempoResolucao.dt.days < 0]

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
167,2016080013232,2016-03-31 22:59:00,2016-01-04 01:47:00,Riscos Tecnológicos / Acidentes / Colisão rodo...,Encerrada,FARO,CASTRO MARIM,Altura,Altura,"37,182018","-7,501312",4.0,9.0,0.0,0.0,-88 days +02:48:00
411,2016100021726,2016-04-26 18:54:00,2016-01-10 01:00:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,LEIRIA,BOMBARRAL,Bombarral e Vale Covo,BOMBARRAL,"39,26731314","-9,171316061",0.0,0.0,0.0,0.0,-108 days +06:06:00
457,2016130063262,2016-04-29 19:40:00,2016-01-10 01:00:00,Riscos Mistos / Incêndios Rurais / Povoamento ...,Encerrada,PORTO,VILA DO CONDE,Retorta e Tougues,TOUGUES,"41,355923","-8,710585",0.0,0.0,0.0,0.0,-111 days +05:20:00
468,2016130063862,2016-04-30 23:40:00,2016-01-05 00:24:00,Riscos Mistos / Incêndios Rurais / Mato,Encerrada,PORTO,PENAFIEL,Perozelo,PEROZELO,"41,155528","-8,260287",3.0,13.0,0.0,0.0,-117 days +00:44:00
469,2016110076737,2016-04-30 22:48:00,2016-01-05 00:24:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,LISBOA,LOURES,Loures,A-dos-Cães,"38,84072474","-9,23072896",1.0,2.0,0.0,0.0,-117 days +01:36:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557694,2018130193923,2018-11-30 22:12:00,2018-01-12 02:30:00,Protecção e Assistência a Pessoas e Bens / Int...,Encerrada,PORTO,BAIÃO,Teixeira e Teixeiró,TEIXEIRÓ,"41,187259","-7,907858",1.0,2.0,0.0,0.0,-323 days +04:18:00
557713,2018140073836,2018-11-30 21:24:00,2018-01-12 00:19:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,SANTARÉM,CARTAXO,Cartaxo e Vale da Pinta,Cartaxo,"39,1637329581298","-8,80302088990241",1.0,2.0,0.0,0.0,-323 days +02:55:00
557717,2018110241552,2018-11-30 21:20:00,2018-01-12 01:02:00,Protecção e Assistência a Pessoas e Bens / Ass...,Encerrada,LISBOA,LISBOA,Benfica,Benfica,"38,744634","-9,19935",1.0,2.0,0.0,0.0,-323 days +03:42:00
557789,2018140073820,2018-11-30 19:41:00,2018-01-12 00:00:00,Riscos Tecnológicos / Acidentes / Colisão rodo...,Encerrada,SANTARÉM,SARDOAL,Alcaravela,SARDOAL,"39,584872","-8,144537",8.0,21.0,0.0,0.0,-323 days +04:19:00


In [78]:
# Removing negative times of resolution
data.drop(data[data.TempoResolucao.dt.days < 0].index, inplace=True)

In [45]:
data.Natureza.isna().sum()
data[data.Natureza.isna()]

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
83933,\t\t\t\t\t\t\t\t,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83934,10.15\tTomar\tIPO\t\t\t\tS\t\tS,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83935,11.30\tTomar\tIPO\t\t\t\tS\t\tN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83936,\t\t\t\t\t\t\t\t,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
83937,13.15\tTomar\tIPO\t\t\t\tS\t\tS,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [46]:
# Droping useless rows
data.dropna(subset=['Natureza'], inplace=True)
data[data.Natureza.isna()]


,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao


In [81]:
data.EstadoOcorrencia.unique()

array(['Encerrada', 'Em Curso', 'Falso Alarme', 'Falso Alerta',
       'Conclusão', 'Despacho', 'Despacho de 1º Alerta', 'Em Resolução',
       'Chegada ao TO', 'Vigilância'], dtype=object)

In [84]:
data[data.EstadoOcorrencia == 'Em Curso']
# Podemos incluir isto para a análise a nível de ocorrências por distrito e assim, mas depois temos que dar drop disto
# temos que discutir isto melhor

,Numero,DataOcorrencia,DataFechoOperacional,Natureza,EstadoOcorrencia,Distrito,Concelho,Freguesia,Localidade,Latitude,Longitude,NumeroMeiosTerrestresEnvolvidos,NumeroOperacionaisTerrestresEnvolvidos,NumeroMeiosAereosEnvolvidos,NumeroOperacionaisAereosEnvolvidos,TempoResolucao
1,2016010098470,2016-10-01 00:30:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,AVEIRO,ALBERGARIA-A-VELHA,Albergaria-a-Velha e Valmaior,Rua 25 de Abril - Assilhó - Albergaria-a-Velha,"40,6927855","-8,479686736",0.0,0.0,0.0,0.0,NaT
3,2016050041560,2016-01-30 23:38:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,CASTELO BRANCO,VILA DE REI,Vila de Rei,Vila de Rei,"39,6684201","-8,146938534",1.0,2.0,0.0,0.0,NaT
119819,2016080056628,2016-12-29 18:56:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,FARO,ALBUFEIRA,Albufeira e Olhos de Água,ALBUFEIRA,"37,09172682","-8,252292264",1.0,2.0,0.0,0.0,NaT
119823,2016090039202,2016-12-29 18:53:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,GUARDA,SEIA,Pinhanços,Lar Pinhanços,"40,46942478","-7,688956785",1.0,2.0,0.0,0.0,NaT
119842,2016080056622,2016-12-29 17:52:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,FARO,OLHÃO,Olhão,OLHÃO,"37,02205442","-7,843132514",1.0,2.0,0.0,0.0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573939,2018150114444,2018-12-31 03:57:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,SETÚBAL,SEIXAL,Amora,Amora,"38,6138797308902","-9,12886309841698",1.0,2.0,0.0,0.0,NaT
573948,2018150114435,2018-12-31 03:18:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,SETÚBAL,GRÂNDOLA,Grândola e Santa Margarida da Serra,GRANDOLA,"38,1825915644394","-8,56071313896018",1.0,2.0,0.0,0.0,NaT
573949,2018100076402,2018-12-31 03:08:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,LEIRIA,LEIRIA,Monte Redondo e Carreira,União das Freguesias de Monte Redondo e Carreira,"39,8947498942488","-8,83974172991636",1.0,2.0,0.0,0.0,NaT
573953,2018150114431,2018-12-31 02:41:00,NaT,Protecção e Assistência a Pessoas e Bens / Ass...,Em Curso,SETÚBAL,ALMADA,"Almada, Cova da Piedade, Pragal e Cacilhas",Cacilhas,"38,6752676774718","-9,1644981889879",0.0,0.0,0.0,0.0,NaT


In [94]:
data[['Natureza']] = data[['Natureza']].astype(str)
# não está a funcionar, tenho que ver isto melhor

In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572877 entries, 0 to 573970
Data columns (total 16 columns):
 #   Column                                  Non-Null Count   Dtype          
---  ------                                  --------------   -----          
 0   Numero                                  572877 non-null  object         
 1   DataOcorrencia                          572877 non-null  datetime64[ns] 
 2   DataFechoOperacional                    417696 non-null  datetime64[ns] 
 3   Natureza                                572877 non-null  object         
 4   EstadoOcorrencia                        572877 non-null  object         
 5   Distrito                                572877 non-null  object         
 6   Concelho                                572877 non-null  object         
 7   Freguesia                               572877 non-null  object         
 8   Localidade                              572877 non-null  object         
 9   Latitude                  

Dados entre os anos 2016 e 2018 - comparar com populção por distrito 

In [ ]:
pd.read_excel('data/pop.xlsx')

,Nome,Divisão Administrativa,População Residente,População/km2,Ano*
0,Lisboa,Distrito Administrativo,2275591,824.2,2021
1,Porto,Distrito Administrativo,1786656,746.0,2021
2,Setúbal,Distrito Administrativo,875656,172.9,2021
3,Braga,Distrito Administrativo,846515,312.8,2021
4,Aveiro,Distrito Administrativo,700964,250.5,2021
5,Faro,Distrito Administrativo,467495,94.6,2021
6,Leiria,Distrito Administrativo,458679,130.9,2021
7,Santarém,Distrito Administrativo,425431,67.9,2021
8,Coimbra,Distrito Administrativo,408631,103.5,2021
9,Viseu,Distrito Administrativo,351592,70.2,2021


## Useful snippet to use later when dealing with training the model

In [59]:
# Convert to time in seconds to use as regression label
x = data.TempoResolucao
y = x.dt.total_seconds()
# Convert back to datetime object
z = pd.to_timedelta(y, unit='s')
z


0        0 days 03:28:00
1                    NaT
2        0 days 00:56:00
3                    NaT
4        0 days 01:25:00
               ...      
573966   0 days 00:53:00
573967   0 days 01:41:00
573968   0 days 02:48:00
573969   0 days 01:46:00
573970   0 days 23:59:00
Name: TempoResolucao, Length: 573964, dtype: timedelta64[ns]